In [11]:
import pandas as pd
df = pd.read_csv("course_catalog.csv")
courses_list = list(df["course_name"])
semesters_list = [1,2,3,4,5,6,7,8]
credits_dict = df.set_index("course_name")["credits"].to_dict()

max_credits = int(input("Enter max credits:"))
min_credits = int(input("Enter min credits:"))

from ortools.sat.python import cp_model 

#building model
model = cp_model.CpModel()

#creating decision variables(each variable has a domain{0,1})
x = {}
for c in courses_list:
    for s in semesters_list:
        x[(c,s)] = model.new_bool_var(f"x_{c}_{s}")

In [12]:
#Hard Constraint 1(each course must be taken atmost once)
for c in courses_list:
    model.Add(sum(x[(c,s)] for s in semesters_list) <= 1)

#Hard Constraint 2(required courses for a particular major must be taken exactly once)
df2 = pd.read_csv("major_requirements.csv")
majors_list = df2["major"].to_list()
for index,value in enumerate(majors_list):
    print(index +1,value)
q = int(input("your selected option is:"))
major_pref = majors_list[q-1]

req_string = df2[df2["major"] == major_pref]["required_courses"].values[0]
for c in req_string.split("_") :
    model.Add(sum(x[(c,s)] for s in semesters_list) == 1)

1 Biochemical_Engineering
2 Biomedical_Engineering
3 Ceramic_Engineering
4 Chemical_Engineering
5 Civil_Engineering
6 ComputerScience_Engineering
7 Electrical_Engineering
8 Electronics_Engineering
9 Engineering_physics
10 Industrial_chemistry
11 MaterialScienceand_Technology
12 Mathematics_and_Computing
13 Mechanical_Engineering
14 Metallurgical_Engineering
15 Mining_Engineering
16 Pharmaceutical_Engineering


In [13]:

#Hard Constraint 3(credit limits)
total_credits = sum(x[(c,s)]*int(credits_dict[c]) for c in courses_list)
for s in semesters_list:
    model.Add(total_credits <= max_credits)
for s in semesters_list:
    model.Add(total_credits >= min_credits)

In [14]:
#Hard Constraint 4(prerequisites)
new_df = df[df["prerequisites"] != "NONE"]
prereq_dict = new_df.set_index("course_name")["prerequisites"].to_dict()
for k in prereq_dict:
    prereq_dict[k] = prereq_dict[k].split("_")

for key in prereq_dict.keys():
    for p in prereq_dict[key]:
        for s in semesters_list:
            model.Add(x[key,s]<= sum(x[p,t] for t in range(1,s)))

In [15]:
#Hard Constraint 5(Time conflicts)
def normalize_and_pad(slot_str):
    # Split the slot string into individual slots (ignore "NONE")
    if slot_str == "NONE":
        slot_list = []
    else:
        slot_list = [slot.split("_") for slot in slot_str.split("/")]

    # Normalize slot to 3 elements each
    normalized = [
        tuple(parts + [None] * (3 - len(parts)))   # pad inside tuple
        for parts in slot_list
    ]

    # Pad L/T/P to always have exactly 3 slots
    while len(normalized) < 3:
        normalized.append((None, None, None))

    # Trim in case there are more than 3 slots
    return tuple(normalized[:3])


y = {}

for c in courses_list:
    row = df[df["course_name"] == c].iloc[0]

    y[c] = {
        "l": normalize_and_pad(row["lecture"]),
        "t": normalize_and_pad(row["tutorial"]),
        "p": normalize_and_pad(row["practical"])
    }
    
LTP = ("l", "t", "p")

for a in courses_list: 
    for b in courses_list: 
        for i in range(3): 
            for j in range(3): 
                for k in LTP: 
                    for m in LTP: 
                        if y[a][m][i][0] == y[b][k][j][0] and m!=k and i!=j and a!=b and y[a][m][i][0] != None: 
                            if y[a][m][i][1] < y[b][k][j][2] or y[b][k][j][1] < y[a][m][i][2] or y[a][m][i][1] == y[b][k][j][1]: 
                                model.Add(x[a,s] + x[b,s] <= 1)

In [16]:
#Hard Constraint 5(Electives)
eldf = pd.read_csv("electives_catalog.csv")

for s in range(5,9):
    model.Add(sum(x[(c,s)] for c in eldf["open_electives"].to_list()) == 1)
for s in range(5,9):
    model.Add(sum(x[c,s] for c in eldf[major_pref].to_list()) == 1)

In [ ]:
#solver
solver = cp_model.CpSolver()
result = solver.Solve(model)


#output
if result in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    print("Optimal Schedule:")
    for s in semesters_list:
        taken = [c for c in courses_list if solver.Value(x[(c, s)]) == 1]
        if taken:
            print(f"Semester {s}: {taken}")
else:
    print("No feasible schedule found.")